In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from itertools import product

In [2]:
# count编码
def count_coding(df, fea_col):
    for f in fea_col:
        df[f + '_count'] = df[f].map(df[f].value_counts())
    return df

# 交叉特征统计
def cross_cat_num(df, num_col, cat_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max',
                '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median'
            })
            df = df.merge(feat, on=f1, how='left')
    return df

In [3]:
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=10, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode
 
        :param n_splits: the number of splits used in mean encoding
 
        :param target_type: str, 'regression' or 'classification'
 
        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """
 
        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}
 
        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None
 
        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
 
    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()
 
        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()
 
        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg(['mean', 'size'])
        col_avg_y['size'] = prior_weight_func(col_avg_y['size'])
        col_avg_y[nf_name] = col_avg_y['size'] * prior + (1 - col_avg_y['size']) * col_avg_y['mean']
        col_avg_y.drop(['size', 'mean'], axis=1, inplace=True)
 
        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values
 
        return nf_train, nf_test, prior, col_avg_y
 
    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)
 
        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new
 
    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
 
        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
 
        return X_new

In [4]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

In [6]:
train_user.head()

,phone_no_m,city_name,county_name,idcard_cnt,label,arpu_202004
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,0,45.000000
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,0,60.000000
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,0,63.000000
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,0,23.203125
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,0,50.000000


In [5]:
test_user.head()

,phone_no_m,city_name,county_name,idcard_cnt,arpu_202004
0,22d522340df77e2252c1a4d92b4bcb00d515e36f3ec6bf...,成都,金牛分公司,1,9.00
1,5220d4b8429bdba3971a7b46a088c6b8fa6710f4060759...,天府新区,天府直属部门,1,NaN
2,7d19dd2b50ced56f03d23bf928cf34dc570a48525571a8...,成都,锦江分公司,1,9.00
3,134a4a591185c9d3788021896dcfc235e9e0a6a1e3f8a4...,绵阳,江油分公司,1,138.75
4,2356dcd6759d50455ddaeed03c838843558e9182d5962f...,成都,青羊分公司,2,39.00


In [7]:
train_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6106 entries, 0 to 6105
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   phone_no_m   6106 non-null   object 
 1   city_name    6106 non-null   object 
 2   county_name  6106 non-null   object 
 3   idcard_cnt   6106 non-null   int8   
 4   label        6106 non-null   int8   
 5   arpu_202004  5369 non-null   float16
dtypes: float16(1), int8(2), object(3)
memory usage: 374.7+ KB


In [8]:
test_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2045 entries, 0 to 2044
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   phone_no_m   2045 non-null   object 
 1   city_name    2045 non-null   object 
 2   county_name  2045 non-null   object 
 3   idcard_cnt   2045 non-null   int8   
 4   arpu_202004  1869 non-null   float16
dtypes: float16(1), int8(1), object(3)
memory usage: 149.9+ KB


In [9]:
df_user = pd.concat([train_user, test_user])

In [10]:
# 号码量*消费值
df_user['idcard_cnt*arpu_202004'] = df_user['idcard_cnt'] * df_user['arpu_202004']

In [11]:
# count编码
count_list = ['city_name', 'county_name', 'idcard_cnt']

df_user = count_coding(df_user, count_list)

In [12]:
cross_cat = ['city_name', 'county_name']
cross_num = ['idcard_cnt']
data = cross_cat_num(df_user, cross_num, cross_cat)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.72it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 57.84it/s]


In [13]:
train_voc = pd.read_hdf('../input/train_voc.h5')
test_voc = pd.read_hdf('../input/test_voc.h5')

In [14]:
train_voc.head()

,phone_no_m,opposite_no_m,calltype_id,start_datetime,call_dur,city_name,county_name,imei_m
9,b3994b4c46e39954cfe0cb8ebd2a79703a2ace6612fa18...,1b01124a6ad0443ad868757c6594a605a0c5179c2b5b57...,1,2020-03-22 21:38:40,175,nan,nan,90ba61111dad175330029e10da8264a86dfdb3a281ccb8...
10,b3994b4c46e39954cfe0cb8ebd2a79703a2ace6612fa18...,1b01124a6ad0443ad868757c6594a605a0c5179c2b5b57...,1,2020-03-23 18:05:41,33,nan,nan,90ba61111dad175330029e10da8264a86dfdb3a281ccb8...
219,c5502a6d3e4cabaf8c0e298c4aac693b6e80835d50d4e4...,f1b73c46a691c068bfd38253e20e882d4993c8d3e58016...,1,2020-03-01 11:24:37,39,nan,nan,e1e1149b2c06e9972293c457293abb32ad05b0c82b3b64...
222,c5502a6d3e4cabaf8c0e298c4aac693b6e80835d50d4e4...,5bc6d97a3017578edfa7979fa43653cd74bad827f18337...,1,2020-03-01 11:23:44,32,nan,nan,e1e1149b2c06e9972293c457293abb32ad05b0c82b3b64...
276,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,c42eeddeff98feb87860441ea6548f8bda2a080c935e02...,2,2020-03-03 15:34:21,3,nan,nan,3e808a14fa2aa61524c70f262539d1c8532b8e3a8c7c87...


In [15]:
test_voc.head()

,phone_no_m,opposite_no_m,calltype_id,start_datetime,call_dur,city_name,county_name,imei_m
0,b3dce36871f3e88164b18d4953b114163f008cb51c28fe...,f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2...,1,2020-04-13 21:04:10,1909,nan,nan,685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f...
1,b3dce36871f3e88164b18d4953b114163f008cb51c28fe...,f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2...,1,2020-04-13 18:32:50,2510,nan,nan,685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f...
2,b3dce36871f3e88164b18d4953b114163f008cb51c28fe...,f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2...,1,2020-04-13 19:55:12,25,nan,nan,685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f...
3,7ec68a368fbbec3279a6a34847f7959623dbff4638351a...,ed8280e5590e40e35a2af7d8708a0754540b0612600d52...,1,2020-04-20 14:11:44,276,nan,nan,a60e82a4383faa79ea972adc48686a729f689b7d1923bc...
4,7ec68a368fbbec3279a6a34847f7959623dbff4638351a...,ba792e76c3e7ee47d2403083f6def313372ae450ddf0b9...,2,2020-04-20 12:56:33,539,nan,nan,a60e82a4383faa79ea972adc48686a729f689b7d1923bc...


In [16]:
train_voc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709144 entries, 9 to 5015413
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   phone_no_m      709144 non-null  object
 1   opposite_no_m   709144 non-null  object
 2   calltype_id     709144 non-null  int8  
 3   start_datetime  709144 non-null  object
 4   call_dur        709144 non-null  int16 
 5   city_name       709144 non-null  object
 6   county_name     709144 non-null  object
 7   imei_m          709144 non-null  object
dtypes: int16(1), int8(1), object(6)
memory usage: 59.9+ MB


In [17]:
test_voc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276522 entries, 0 to 276521
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   phone_no_m      276522 non-null  object
 1   opposite_no_m   276522 non-null  object
 2   calltype_id     276522 non-null  int8  
 3   start_datetime  276522 non-null  object
 4   call_dur        276522 non-null  int16 
 5   city_name       276522 non-null  object
 6   county_name     276522 non-null  object
 7   imei_m          276522 non-null  object
dtypes: int16(1), int8(1), object(6)
memory usage: 25.6+ MB


In [18]:
train_voc.nunique()

phone_no_m          4823
opposite_no_m     281103
calltype_id            3
start_datetime    561615
call_dur            2653
city_name             22
county_name          193
imei_m              6025
dtype: int64

In [19]:
test_voc.nunique()

phone_no_m          1965
opposite_no_m     107683
calltype_id            3
start_datetime    253144
call_dur            2089
city_name             22
county_name          173
imei_m              2656
dtype: int64

In [20]:
df_voc = pd.concat([train_voc, test_voc])

In [23]:
df_voc['voc_day'] = df_voc['start_datetime'].astype('datetime64').dt.day
df_voc['voc_hour'] = df_voc['start_datetime'].astype('datetime64').dt.hour
df_voc['voc_dayofweek'] = df_voc['start_datetime'].astype('datetime64').dt.dayofweek

In [24]:
df_voc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985666 entries, 9 to 276521
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   phone_no_m      985666 non-null  object
 1   opposite_no_m   985666 non-null  object
 2   calltype_id     985666 non-null  int8  
 3   start_datetime  985666 non-null  object
 4   call_dur        985666 non-null  int16 
 5   city_name       985666 non-null  object
 6   county_name     985666 non-null  object
 7   imei_m          985666 non-null  object
 8   voc_day         985666 non-null  int64 
 9   voc_hour        985666 non-null  int64 
 10  voc_dayofweek   985666 non-null  int64 
dtypes: int16(1), int64(3), int8(1), object(6)
memory usage: 118.0+ MB


In [25]:
# 每天通话次数统计
df_voc['voc_day_count'] = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].transform('count')
df_voc['voc_day_count_max'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('max')
df_voc['voc_day_count_min'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('min')
df_voc['voc_day_count_mean'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('mean')
df_voc['voc_day_count_std'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('std')

del df_voc['voc_day_count']

In [28]:
df_voc[['phone_no_m', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std']].head(30)

,phone_no_m,voc_day_count_max,voc_day_count_min,voc_day_count_mean,voc_day_count_std
9,b3994b4c46e39954cfe0cb8ebd2a79703a2ace6612fa18...,9,1,6.248000,2.415194
10,b3994b4c46e39954cfe0cb8ebd2a79703a2ace6612fa18...,9,1,6.248000,2.415194
219,c5502a6d3e4cabaf8c0e298c4aac693b6e80835d50d4e4...,5,1,2.500000,1.638356
222,c5502a6d3e4cabaf8c0e298c4aac693b6e80835d50d4e4...,5,1,2.500000,1.638356
276,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,10,1,5.051282,2.997390
290,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,10,1,5.051282,2.997390
291,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,10,1,5.051282,2.997390
292,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,10,1,5.051282,2.997390
296,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,10,1,5.051282,2.997390
297,2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39...,10,1,5.051282,2.997390


In [ ]:
# 总的通话次数
df_voc['voc_count'] = df_voc.groupby('phone_no_m')['phone_no_m'].transform('count')

# 相互打电话次数
df_voc['voc_count_mutual'] = df_voc.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform('count')
